<a href="https://colab.research.google.com/github/Baldros/CALIPSO/blob/main/ExtractTables_LV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação:

    O objetivo desse código é, em primeira momento, extrair as informações do conjunto
    de dados como tabelas, de modo a facilitar a construção de gráficos.

    E, em segundo momento, a ideia é criar conexão com um SGBD de modo a armazenar essas
    tabelas de deixa-las guardadas para que outros possam ter acesso.

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from time import time
from pyhdf.SD import SD, SDC
from processamento_dados import (selecao_area,
                                 controle_qualidade, porcentagem_valida,
                                 calc_media_espacial,
                                 calc_media_sazonal_1d, calc_media_sazonal_2d,
                                 calc_media_sazonal_anual_1d, calc_media_sazonal_anual_2d,
                                 convert_dicionario_1d_dataframe, convert_dicionario_2d_dataframe,
                                 preenchendo_dados,  calc_soma_deteccoes_coluna,hdf_path)

# Extraindo informação:

    Temos dois códigos de LV3, a primeira versão é mais arcaico e a segundo versão é mais
    bem elaborado. A principio, vamos tentar extrair as tabelas utilizadas do LV3 mais bem
    elaborado e torcer para ele vir bem organizado.

In [ ]:
# Defindo os diretórios:
dir1 = r'D:\Estudo\LASA\Dados\Bahia\L2_VFM'
dir2 = r'D:\Estudo\LASA\Dados\Bahia\L3_Total'

In [ ]:
# Inervalo de Tempo:
ano_inicial = input('Data de Inicio(Ano): ')
ano_final = input('Data de Final(Ano): ')


if ano_inicial == '':
    ano_inicial = '2006'

elif int(ano_inicial) < 2006:
    print('Não existe valores anteriores a 2006')


if ano_final == '':
    ano_final = '2022'

elif int(ano_final) > 2022:
    print('Não existe valores maiores que 2022')

files = hdf_path(dir2, [ano_inicial,ano_final])

Data de Inicio(Ano):  
Data de Final(Ano):  


In [ ]:
# Lendo o arquivo JSON
with open('info.json', 'r') as file:
    json_data = file.read()

# Convertendo a string JSON de volta para um dicionário
info = json.loads(json_data);info

{'Produtos': {'Vertical_Feature_Mask': {'Diretório': 'D:\\Estudo\\LASA\\Dados\\Bahia\\L2_VFM',
   'Classificacao_elementos': {'Tipos_elementos': {'Bits': {'0': 'Inválido',
      '1': 'Ar limpo',
      '2': 'Nuvem',
      '3': 'Aerossol troposférico',
      '4': 'Aerossol estratosférico',
      '5': 'Superfície',
      '6': 'Subsuperfície',
      '7': 'Sem sinal'}},
    'Aerossóis_Troposféricos': {'Bits': {'0': 'Não determinado',
      '1': 'Marinho Limpo',
      '2': 'Poeira',
      '3': 'Poluição Continental/Fumaça',
      '4': 'Continental Limpo',
      '5': 'Poeira Poluída',
      '6': 'Fumaça Elevada',
      '7': 'Poeira Marinha'}},
    'Aerossóis_Estratosféricos': {'Bits': {'0': 'Inválido',
      '1': 'Aerosol PSC',
      '2': 'Cinza Vulcânica',
      '3': 'Sulfato/Outro',
      '4': 'Fumaça Elevada'}}}},
  'Tropospheric_Aerosol_Profile': {'Diretório': 'D:\\Estudo\\LASA\\Dados\\Bahia\\L3_Total',
   'Variáveis': {'Extinction_Coefficient_532_Mean': {'Tradução': 'Coeficiente de Extin

**Processando os dados**

In [ ]:
# Variável contendo informações do produto de perfil de aerossóis troposféricos
info_produto = info['Produtos']['Tropospheric_Aerosol_Profile']

#files = [f.path for f in os.scandir(info_produto['Diretório'])]

niveis_altitude = SD(files[0], SDC.READ).select('Altitude_Midpoint').get()[0]
perfil_media_sazonal_anual = {}
perfil_media_sazonal = {}
dados = {}
for area, info_area in list(info['Áreas_estudo'].items()):
    # Cria um dicionário vazio para cada área
    perfil_media_sazonal_anual[area] = {}
    perfil_media_sazonal[area] = {}
    dados[area] = {}
    for variavel, info_variavel in info_produto["Variáveis"].items():
        # Cria uma chave no dicionário para cada variável
        dados[area][variavel] = {}
        coordenadas = info_area['Coordenadas']
        for file in files:
            dado = SD(file, SDC.READ)
            # Abre cada arquivo listado
            # Seleciona dados de cada variável fazendo corte de área
            dado = selecao_area(dado, coordenadas, variavel)
            dado = controle_qualidade(dado, info_variavel['Limites_detecção'])
            # Adicionar mês ao dicionário apenas quando 80% dos dados passam no CQ
            if porcentagem_valida(dado) >= 0.8:
                dados[area][variavel][file[-12:-5]] = calc_media_espacial(dado)

        if variavel == 'Aerosol_Type':
            # Transforma array contendo classificação dos aerossóis em dataframe com colunas correspondendo a cada tipo de aerossol
            dados[area][variavel] = convert_dicionario_2d_dataframe(dados[area][variavel], info_variavel)
            # Salvando df com médias sazonais para cada ano e tipo de aerossol na chave da variável de detecção de aerossóis por tipo
            perfil_media_sazonal_anual[area][variavel] = calc_media_sazonal_anual_2d(dados[area][variavel])
            perfil_media_sazonal[area][variavel] = calc_media_sazonal_2d(perfil_media_sazonal_anual[area][variavel])
        else:
            # Salvando df com as médias sazonais para cada ano de cada variável de perfil único na chave da variável
            perfil_media_sazonal_anual[area][variavel] = calc_media_sazonal_anual_1d(dados[area][variavel])
            # Salvando df com as médias sazonais do período para cada variável de perfil único na chave da variável
            perfil_media_sazonal[area][variavel] = calc_media_sazonal_1d(perfil_media_sazonal_anual[area][variavel])

        if 'Ext' not in variavel:
            # Calcula a soma das detecções para a coluna
            dados[area][variavel] = calc_soma_deteccoes_coluna(dados[area][variavel])
            # Tranformando dicionário em df
            dados[area][variavel] = convert_dicionario_1d_dataframe(dados[area][variavel])
            dados[area][variavel] = preenchendo_dados(dados[area][variavel])

    dados[area]['AOD'] = pd.concat({variavel: valor for variavel, valor in dados[area].items() if 'AOD' in variavel}, axis=1).droplevel(1, axis=1)


C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)
C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)
C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)
C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)


**Extraindo a Informação**

In [ ]:
dados.keys()

dict_keys(['Roi_Atlantico'])

In [ ]:
dados['Roi_Atlantico'].keys()

dict_keys(['Extinction_Coefficient_532_Mean', 'Extinction_Coefficient_532_Mean_Dust', 'Extinction_Coefficient_532_Mean_Elevated_Smoke', 'Extinction_Coefficient_532_Mean_Polluted_Dust', 'AOD_Mean', 'AOD_Mean_Dust', 'AOD_Mean_Elevated_Smoke', 'AOD_Mean_Polluted_Dust', 'Samples_Aerosol_Detected_Accepted', 'Aerosol_Type', 'AOD'])

In [ ]:
dict_dados = {}

for key in dados['Roi_Atlantico'].keys():
    print(key)
    display(pd.DataFrame(dados['Roi_Atlantico'][key]))
    dict_dados[key] = pd.DataFrame(dados['Roi_Atlantico'][key])

Extinction_Coefficient_532_Mean


,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,...,2021-02,2021-03,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002398,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.001549,0.002081
204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003275,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.001812,0.001914
205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002666,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.001411,0.006079
206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001515,0.0,0.0,0.0,0.0,0.0,0.001442,0.0,0.001352,0.006054


Extinction_Coefficient_532_Mean_Dust


,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,...,2021-02,2021-03,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001549,0.002081
204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001812,0.001914
205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001411,0.006079
206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001352,0.006054


Extinction_Coefficient_532_Mean_Elevated_Smoke


...

,0
2006-06,4097.0
2006-07,7403.0
2006-08,7343.0
2006-09,7934.5
2006-10,3359.5
...,...
2021-07,582.5
2021-08,255.0
2021-09,114.0
2021-10,324.0


Aerosol_Type


,Marinho Limpo,Poeira,Poluição Continental/Fumaça,Continental Limpo,Poeira Poluída,Fumaça Elevada,Poeira Marinha
2006-06,3858.0,0.0,42.5,0.0,0.0,0.0,196.5
2006-07,5790.0,139.0,159.0,0.0,0.0,456.0,859.0
2006-08,5071.0,329.0,246.5,0.0,0.0,723.5,973.0
2006-09,5001.0,159.0,12.0,0.0,163.5,2184.0,415.0
2006-10,2677.5,98.0,0.0,0.0,0.0,78.5,505.5
...,...,...,...,...,...,...,...
2021-07,146.5,356.0,0.0,0.0,0.0,39.5,40.5
2021-08,62.5,153.0,0.0,0.0,0.0,15.0,24.5
2021-09,18.0,51.0,0.0,0.0,0.0,24.0,21.0
2021-10,35.5,35.5,0.0,0.0,0.0,238.5,14.5


AOD


,AOD_Mean,AOD_Mean_Dust,AOD_Mean_Elevated_Smoke,AOD_Mean_Polluted_Dust
2006-06,0.110947,0.000000,0.000000,0.000000
2006-07,0.065967,0.001535,0.002769,0.000000
2006-08,0.080245,0.003798,0.005631,0.000000
2006-09,0.123775,0.002018,0.020964,0.001155
2006-10,0.039216,0.001596,0.001464,0.000000
...,...,...,...,...
2021-07,0.013541,0.008726,0.001802,0.000000
2021-08,0.011039,0.009530,0.000226,0.000000
2021-09,0.002509,0.001286,0.000332,0.000000
2021-10,0.031243,0.003092,0.024649,0.000000


    Show, temos a nossa informação, agora é só organiza-la e começar para ver como será feito o armazenamento
    desse processo.

# Tratando informações:

    Algumas tabelas estão com a data como index. Não queremos isso dessa forma, porque
    na hora de passar para o SGBD pode ser que percamos essa informação. Então vamos
    tratar esses dados para extrair essa informação da melhor forma possivel.

In [ ]:
# Tabelas limpas:
limpas = ['Extinction_Coefficient_532_Mean','Extinction_Coefficient_532_Mean_Dust',
          'Extinction_Coefficient_532_Mean_Elevated_Smoke','Extinction_Coefficient_532_Mean_Polluted_Dust']

sujas = dict_dados.keys() - limpas

for key in sujas:
    print(key)
    df = dict_dados[key].reset_index().rename(columns={'index': 'Data'})
    #df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m')
    #df['Data'] = df['Data'].dt.to_period('M')
    dict_dados[key] = df
    display(df)

AOD


,Data,AOD_Mean,AOD_Mean_Dust,AOD_Mean_Elevated_Smoke,AOD_Mean_Polluted_Dust
0,2006-06,0.110947,0.000000,0.000000,0.000000
1,2006-07,0.065967,0.001535,0.002769,0.000000
2,2006-08,0.080245,0.003798,0.005631,0.000000
3,2006-09,0.123775,0.002018,0.020964,0.001155
4,2006-10,0.039216,0.001596,0.001464,0.000000
...,...,...,...,...,...
181,2021-07,0.013541,0.008726,0.001802,0.000000
182,2021-08,0.011039,0.009530,0.000226,0.000000
183,2021-09,0.002509,0.001286,0.000332,0.000000
184,2021-10,0.031243,0.003092,0.024649,0.000000


AOD_Mean_Dust


,Data,0
0,2006-06,0.000000
1,2006-07,0.001535
2,2006-08,0.003798
3,2006-09,0.002018
4,2006-10,0.001596
...,...,...
181,2021-07,0.008726
182,2021-08,0.009530
183,2021-09,0.001286
184,2021-10,0.003092


AOD_Mean_Elevated_Smoke


...

,Data,0
0,2006-06,0.110947
1,2006-07,0.065967
2,2006-08,0.080245
3,2006-09,0.123775
4,2006-10,0.039216
...,...,...
181,2021-07,0.013541
182,2021-08,0.011039
183,2021-09,0.002509
184,2021-10,0.031243


Aerosol_Type


,Data,Marinho Limpo,Poeira,Poluição Continental/Fumaça,Continental Limpo,Poeira Poluída,Fumaça Elevada,Poeira Marinha
0,2006-06,3858.0,0.0,42.5,0.0,0.0,0.0,196.5
1,2006-07,5790.0,139.0,159.0,0.0,0.0,456.0,859.0
2,2006-08,5071.0,329.0,246.5,0.0,0.0,723.5,973.0
3,2006-09,5001.0,159.0,12.0,0.0,163.5,2184.0,415.0
4,2006-10,2677.5,98.0,0.0,0.0,0.0,78.5,505.5
...,...,...,...,...,...,...,...,...
181,2021-07,146.5,356.0,0.0,0.0,0.0,39.5,40.5
182,2021-08,62.5,153.0,0.0,0.0,0.0,15.0,24.5
183,2021-09,18.0,51.0,0.0,0.0,0.0,24.0,21.0
184,2021-10,35.5,35.5,0.0,0.0,0.0,238.5,14.5


AOD_Mean_Polluted_Dust


,Data,0
0,2006-06,0.000000
1,2006-07,0.000000
2,2006-08,0.000000
3,2006-09,0.001155
4,2006-10,0.000000
...,...,...
181,2021-07,0.000000
182,2021-08,0.000000
183,2021-09,0.000000
184,2021-10,0.000000


In [ ]:
dict_dados['Aerosol_Type'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Data                         186 non-null    object 
 1   Marinho Limpo                138 non-null    float64
 2   Poeira                       138 non-null    float64
 3   Poluição Continental/Fumaça  138 non-null    float64
 4   Continental Limpo            138 non-null    float64
 5   Poeira Poluída               138 non-null    float64
 6   Fumaça Elevada               138 non-null    float64
 7   Poeira Marinha               138 non-null    float64
dtypes: float64(7), object(1)
memory usage: 11.8+ KB


In [ ]:
lista_2col = []
for key in sujas:
    if len(dict_dados[key].columns) <= 2:
        print(key)
        dict_dados[key].rename(columns={0:'Valores'},inplace=True)
        display(dict_dados[key].info())
        lista_2col.append(key)
        print()

AOD_Mean_Dust
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Data     186 non-null    object 
 1   Valores  138 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


None


AOD_Mean_Elevated_Smoke
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Data     186 non-null    object 
 1   Valores  136 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


None


Samples_Aerosol_Detected_Accepted
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Data     186 non-null    object 
 1   Valores  138 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


None


AOD_Mean
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Data     186 non-null    object 
 1   Valores  138 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


None


AOD_Mean_Polluted_Dust
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Data     186 non-null    object 
 1   Valores  137 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


None

    Poderia pensar em dividir a coluna Data em duas colunas, mês e ano, porém
    não sei muito bem como acomodar isso no MySQL, então vou deixar assim
    por enquanto.

# Armazendo os dados:

    Passado a etapa de processamento dos dados, agora o objetivo é como armazena-los. Para tal, o que eu pretendo
    fazer aqui é armazenar essas tabelas num SGBD. Por conta de familiaridade, vou armazenar no MySQL, que acredito
    ser o mais fácil de se comunicar com o python.

In [ ]:
!pip install mysql-connector-python

**Constuindo um usuário no MySQL**

    Se faz interessante que você já tenha criado um usuário no SGBD para que você
    possa conectar. O processo de criar usuário é simples:

    1. Entre no prompt (cmd do windows):
    utilize o comando mysql -u root -p e insira o password que por
    padrão é root.

    2. Insira os comandos:
    CREATE USER 'seu_nome_de_usuario'@'localhost' IDENTIFIED BY 'sua_senha';
    GRANT ALL PRIVILEGES ON *.* TO 'seu_nome_de_usuario'@'localhost';
    FLUSH PRIVILEGES;

    Pronto, usuário criado. Você então precisará do usuário e da senha
    para conectar-se ao SGBD.

In [ ]:
import mysql.connector

In [ ]:
# Conectando-se ao SGBD:
mydb = mysql.connector.connect(
  host="localhost",
  user="bd_project",
  password="*****"
)

if mydb.is_connected() == True:
    print('Conectado ao SGBD')

Conectado ao SGBD


In [ ]:
# Instanciando o cursor para transferência de informação:
cursor = mydb.cursor()

    O cursor é o objeto que executa os processos entre o python
    e o SGBD, ou seja, é quem realmente faz as coisas acontecerem.

    Vale ressaltar que a linguagem SQL não sobrescreve os comandos,
    ou seja, certifique-se de estar escrevendo as coisas corretamente,
    se não se terá que apagar a tabela e fazer de novo.

In [ ]:
# Criando Schema:
cursor.execute("CREATE DATABASE IF NOT EXISTS aerossol_database")
cursor.execute("USE aerossol_database")

**Criando as Tabelas e Inserindo os Dados**

    A ideia aqui é analisar cada dataframe para construir as tabelas
    no SGBD de forma otimizada, ou seja, aqueles dataframes de mesma
    estrutura serão agrupados e as tabelas no SGBD serão criadas juntas.

In [ ]:
def insert_data(dataframe, tabela):
    '''
        Função construida para inserir
    dados numa tabela qualquer.
    '''

    # Ajustando valores nan para serem inseridos na tabela:
    dataframe.replace({'nan': None, np.nan: None}, inplace=True)

    # Extraindo as colunas como string:
    colunas = ', '.join([f'`{col}`' for col in dataframe.columns])

    inicio = time()

    # Preparando os valores para inserção
    values = [tuple(row) for row in dataframe.to_numpy()]

    # Construindo a instrução SQL com %s para cada valor
    insert = f"INSERT INTO {tabela} ({colunas}) VALUES ({', '.join(['%s'] * len(dataframe.columns))})"

    # Executando a inserção em lote
    cursor.executemany(insert, values)
    mydb.commit()

    final = time()

    print(f'Dados inseridos com sucesso na tabela {var}')
    print('Tempo de processamento:', int(final - inicio), 'segundos')

In [ ]:
# Criando tabelas de Coeficiente de Extinção:
for var in limpas:
    colunas = ', '.join([f'`{col}` FLOAT' for col in dict_dados[var]])  # Substitua `DECIMAL(10, 6)` pelo tipo de dado apropriado
    cursor.execute(
        f'''
        CREATE TABLE IF NOT EXISTS {var} (
            id INT AUTO_INCREMENT PRIMARY KEY,
            {colunas}
        );
        '''
    )

In [ ]:
# Inserindo dados:
for key in limpas:
    insert_data(dict_dados[key], key)

Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos
Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos
Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos
Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos


In [ ]:
for key in lista_2col:
    cursor.execute(
        f'''
        CREATE TABLE IF NOT EXISTS {key} (
        Data VARCHAR(20),
        Valores FLOAT
        )
        '''
    )

In [ ]:
# Inserindo dados:
for key in lista_2col:
    insert_data(dict_dados[key], key)

Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos
Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos
Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos
Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos
Dados inseridos com sucesso na tabela Extinction_Coefficient_532_Mean_Polluted_Dust
Tempo de processamento: 0 segundos


In [ ]:
# Construindo Tabelas:
cursor.execute(
    f'''
    CREATE TABLE IF NOT EXISTS Aerosol_Type (
        id INT AUTO_INCREMENT PRIMARY KEY,
        Data VARCHAR(20),
        Marinho_Limpo FLOAT,
        Poeira FLOAT,
        Poluicao_Continental_Fumaca FLOAT,
        Continental_Limpo FLOAT,
        Poeira_Poluida FLOAT,
        Fumaca_Elevada FLOAT,
        Poeira_Marinha FLOAT
    );
    '''
)

In [ ]:
# Inserindo dados:
# Substituir NaN por None no DataFrame
dict_dados['Aerosol_Type'].replace({np.nan: None}, inplace=True)

inicio = time()
for index, row in dict_dados['Aerosol_Type'].iterrows():
    insert = '''INSERT INTO Aerosol_Type (Data, Marinho_Limpo, Poeira,
    Poluicao_Continental_Fumaca,Continental_Limpo,Poeira_Poluida,Fumaca_Elevada,Poeira_Marinha)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'''
    val = (row['Data'],row['Marinho Limpo'],row['Poeira'],
           row['Poluição Continental/Fumaça'],row['Continental Limpo'],row['Poeira Poluída'],
           row['Fumaça Elevada'],row['Poeira Marinha'])
    cursor.execute(insert, val)
    mydb.commit()
final = time()

print('Dados inseridos com sucesso no SGBD')
print('Tempo de processamento: ',int(final-inicio), 'segundos')

Dados inseridos com sucesso no SGBD
Tempo de processamento:  1 segundos


In [ ]:
cursor.execute(
    f'''
    CREATE TABLE IF NOT EXISTS AOD (
        id INT AUTO_INCREMENT PRIMARY KEY,
        Data VARCHAR(20),
        AOD_Mean FLOAT,
        AOD_Mean_Dust FLOAT,
        AOD_Mean_Elevated_Smoke FLOAT,
        AOD_Mean_Polluted_Dust FLOAT
    );
    '''
)

In [ ]:
# Inserindo dados:
dict_dados['AOD'].replace({np.nan: None}, inplace=True)

inicio = time()
for index, row in dict_dados['AOD'].iterrows():
    insert = '''INSERT INTO AOD (Data, AOD_Mean, AOD_Mean_Dust,AOD_Mean_Elevated_Smoke,AOD_Mean_Polluted_Dust)
    VALUES (%s,%s,%s,%s,%s)'''
    val = (row['Data'],row['AOD_Mean'],row['AOD_Mean_Dust'],
           row['AOD_Mean_Elevated_Smoke'],row['AOD_Mean_Polluted_Dust'])
    cursor.execute(insert, val)
    mydb.commit()
final = time()

print('Dados inseridos com sucesso no SGBD')
print('Tempo de processamento: ',int(final-inicio), 'segundos')

Dados inseridos com sucesso no SGBD
Tempo de processamento:  2 segundos
